In [7]:
from itertools import repeat
from multiprocessing import Pool

import censusdata
import censusgeocode as cg
import geopandas
import geopandas as gpd
import numpy as np
import pandas as pd
import requests
from geopy import distance

In [8]:
# Read in the bagel geolocated data

final_bagel_data_geo = pd.read_pickle("final_bagel_data_geo.pkl")

In [10]:
# Use FCC API to extract information for each lat/long and its associated census tract

census_tracts = []

for index, row in final_bagel_data_geo.iterrows():

    lat = pd.to_numeric(row["lat_gmap"])
    lng = pd.to_numeric(row["lng_gmap"])
    url = 'https://geo.fcc.gov/api/census/block/find?latitude={}&longitude={}5&format=json'.format(lat,lng)
    response = requests.get(url)
    data = response.json()["Block"]["FIPS"]
    df = pd.DataFrame({"lng_gmap": [lng], "lat_gmap": [lat], "fips": [data]})
    census_tracts.append(df)

In [143]:
# Concat information about census

census_tracts_clean = pd.concat(census_tracts).drop_duplicates()

In [144]:
# Merge the clean census data

final_bagel_data_geo_clean = final_bagel_data_geo.merge(
    census_tracts_clean, on=["lat_gmap", "lng_gmap"], how="left"
)

final_bagel_data_geo_clean[
    "county_tract_bagel"
] = final_bagel_data_geo_clean.fips.str.slice(start=2, stop=11)

In [145]:
final_bagel_data_geo_clean.head(2)

,name,phone,review_count,price,food_type,rating,address,town,search,loc,formatted_address_gmap,lat_gmap,lng_gmap,fips,county_tract_bagel
0,Atlantic Bagel Company,7189345800,1.0,NaN,Bagels,3 star rating,2 Neptune Ave,Brighton Beach,Atlantic Bagel Company 2 Neptune Ave Brighton ...,"[{'address_components': [{'long_name': '2', 's...","2 Neptune Ave, Brooklyn, NY 11235, USA",40.58,-73.95,360470610043000,047061004
1,Bagel Nest,7188727545,1.0,NaN,Bagels,4 star rating,1237 Fulton St,Bedford Stuyvesant,Bagel Nest 1237 Fulton St Bedford Stuyvesant,"[{'address_components': [{'long_name': '1237',...","1237 Fulton St, Brooklyn, NY 11216, USA",40.68,-73.95,360470245004001,047024500


In [146]:
# Get population estimates from census for 2019

pd.set_option("display.expand_frame_repr", False)
pd.set_option("display.precision", 2)

pop = censusdata.download(
    "acs5",
    2019,
    censusdata.censusgeo([("state", "36"), ("county", "*"), ("tract", "*")]),
    ["B01003_001E", "GEO_ID"],
)
pop = pop[["GEO_ID", "B01003_001E"]].rename(columns={"B01003_001E": "total_pop"})
pop["county_tract"] = pop["GEO_ID"].str.slice(start=11, stop=20)
pop["state_county_tract"] = pop["GEO_ID"].str.slice(start=9, stop=20)
pop_clean = pop.drop(["GEO_ID"], axis=1).reset_index(drop=True)

In [147]:
pop_clean

,total_pop,county_tract,state_county_tract
0,3563,067005500,36067005500
1,1599,067005601,36067005601
2,1842,067006102,36067006102
3,3844,067011201,36067011201
4,3950,067005602,36067005602
...,...,...,...
4913,3048,067001800,36067001800
4914,1393,067003400,36067003400
4915,1387,067004000,36067004000
4916,1541,067004800,36067004800


In [148]:
# Read in census tract to NTA crosswalk

cen_tract_nta = (
    pd.read_excel("nyc2010census_tabulation_equiv.xlsx", skiprows=3, dtype="object")
    .rename(
        columns={
            "Borough": "borough",
            "2010 Census Bureau FIPS County Code": "fips_county",
            "2010 NYC Borough Code": "borough_code",
            "2010 Census Tract": "census_tract",
            "PUMA": "puma",
            "Neighborhood Tabulation Area (NTA)": "nta_code",
            "Unnamed: 6": "nta_name",
        }
    )
    .tail(-1)
    .reset_index(drop=True)
)

In [149]:
# Format to get census tract name

cen_tract_nta['county_tract'] = cen_tract_nta['fips_county'] + cen_tract_nta['census_tract']

In [150]:
# Merge to get the Neighborhood group

cen_tract_nta_clean = (
    final_bagel_data_geo_clean.merge(
        cen_tract_nta, left_on="county_tract_bagel", right_on="county_tract"
    )
    .merge(pop_clean, on="county_tract")
    .drop(["county_tract","state_county_tract"], axis=1)
)

In [151]:
cen_tract_nta_clean.head(2)

,name,phone,review_count,price,food_type,rating,address,town,search,loc,...,fips,county_tract_bagel,borough,fips_county,borough_code,census_tract,puma,nta_code,nta_name,total_pop
0,Atlantic Bagel Company,7189345800,1.0,NaN,Bagels,3 star rating,2 Neptune Ave,Brighton Beach,Atlantic Bagel Company 2 Neptune Ave Brighton ...,"[{'address_components': [{'long_name': '2', 's...",...,360470610043000,047061004,Brooklyn,047,3,061004,4018,BK19,Brighton Beach,6221
1,Bagel Nest,7188727545,1.0,NaN,Bagels,4 star rating,1237 Fulton St,Bedford Stuyvesant,Bagel Nest 1237 Fulton St Bedford Stuyvesant,"[{'address_components': [{'long_name': '1237',...",...,360470245004001,047024500,Brooklyn,047,3,024500,4003,BK75,Bedford,4491


In [152]:
# Extract the rating

cen_tract_nta_clean["rating_num"] = (
    cen_tract_nta_clean["rating"].str.extract("(\d+)").astype("float")
)

In [153]:
# Set any review to NA if review count is less than 5

cen_tract_nta_clean.loc[cen_tract_nta_clean["review_count"] < 5, "rating_num"] = np.nan

In [154]:
cen_tract_nta_clean.head(2)

,name,phone,review_count,price,food_type,rating,address,town,search,loc,...,county_tract_bagel,borough,fips_county,borough_code,census_tract,puma,nta_code,nta_name,total_pop,rating_num
0,Atlantic Bagel Company,7189345800,1.0,NaN,Bagels,3 star rating,2 Neptune Ave,Brighton Beach,Atlantic Bagel Company 2 Neptune Ave Brighton ...,"[{'address_components': [{'long_name': '2', 's...",...,047061004,Brooklyn,047,3,061004,4018,BK19,Brighton Beach,6221,NaN
1,Bagel Nest,7188727545,1.0,NaN,Bagels,4 star rating,1237 Fulton St,Bedford Stuyvesant,Bagel Nest 1237 Fulton St Bedford Stuyvesant,"[{'address_components': [{'long_name': '1237',...",...,047024500,Brooklyn,047,3,024500,4003,BK75,Bedford,4491,NaN


In [155]:
cen_tract_nta_final = (
    cen_tract_nta_clean[["nta_name", "nta_code", "name", "rating_num", "total_pop"]]
    .groupby(["nta_code", "nta_name"])
    .agg({"name": "count", "rating_num": "mean", "total_pop": "sum"})
    .reset_index()
    .rename(columns={'name':'n_bagel_shops'})
)

In [156]:
cen_tract_nta_final.head()

,nta_code,nta_name,n_bagel_shops,rating_num,total_pop
0,BK09,Brooklyn Heights-Cobble Hill,3,3.00,12732
1,BK17,Sheepshead Bay-Gerritsen Beach-Manhattan Beach,7,2.86,26931
2,BK19,Brighton Beach,1,NaN,6221
3,BK23,West Brighton,1,3.00,5765
4,BK25,Homecrest,5,3.50,16830


In [157]:
# Get the bagel per capita estimates

cen_tract_nta_final["bagel_per_capita"] = (
    cen_tract_nta_final["n_bagel_shops"]
    / cen_tract_nta_final["total_pop"]
    * 10000
)

In [158]:
# Save data file 

cen_tract_nta_final.to_csv("nta_cen_bagels_clean.csv", index=False)

In [159]:
# Data: https://www1.nyc.gov/site/planning/data-maps/open-data/dwn-nynta.page
geojson_nyc = gpd.read_file("https://services5.arcgis.com/GfwWNkhOj9bNBqoJ/ArcGIS/rest/services/NYC_Neighborhood_Tabulation_Areas/FeatureServer/0/query?where=1=1&outFields=*&outSR=4326&f=pgeojson")

In [160]:
cen_tract_nta_final["high_rating_flag"] = np.where(
    cen_tract_nta_final.rating_num >= 4, "1", "0"
)

In [161]:
cen_tract_nta_final.loc[
    cen_tract_nta_final.nta_name.str.contains("park-cemetery"), "park_cemetery_flag"
] = 1

In [162]:
cen_tract_nta_final_geo = geojson_nyc.merge(
    cen_tract_nta_final, how="left" , left_on="NTACode", right_on="nta_code"
)

In [163]:
cen_tract_nta_final_geo["n_bagel_shops"] = np.where(
    cen_tract_nta_final_geo.n_bagel_shops.isnull(), 0, cen_tract_nta_final_geo.n_bagel_shops
)

In [164]:
cen_tract_nta_final_geo

,OBJECTID,BoroCode,BoroName,CountyFIPS,NTACode,NTAName,Shape__Area,Shape__Length,geometry,nta_code,nta_name,n_bagel_shops,rating_num,total_pop,bagel_per_capita,high_rating_flag,park_cemetery_flag
0,1,3,Brooklyn,047,BK88,Borough Park,5.40e+07,39247.23,"POLYGON ((-73.97605 40.63128, -73.97717 40.630...",BK88,Borough Park,1.0,3.00,4008.0,2.50,0,NaN
1,2,4,Queens,081,QN51,Murray Hill,5.25e+07,33266.90,"POLYGON ((-73.80379 40.77562, -73.80099 40.775...",QN51,Murray Hill,3.0,3.67,11473.0,2.61,0,NaN
2,3,4,Queens,081,QN27,East Elmhurst,1.97e+07,19816.68,"POLYGON ((-73.86110 40.76367, -73.85993 40.762...",QN27,East Elmhurst,1.0,NaN,2178.0,4.59,0,NaN
3,4,4,Queens,081,QN07,Hollis,2.29e+07,20976.34,"POLYGON ((-73.75726 40.71815, -73.75589 40.716...",NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN
4,5,3,Brooklyn,047,BK25,Homecrest,3.00e+07,27514.02,"POLYGON ((-73.95859 40.61041, -73.95828 40.608...",BK25,Homecrest,5.0,3.50,16830.0,2.97,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
190,191,3,Brooklyn,047,BK93,Starrett City,1.16e+07,17414.58,"MULTIPOLYGON (((-73.88829 40.64673, -73.88829 ...",BK93,Starrett City,1.0,3.00,6744.0,1.48,0,NaN
191,192,3,Brooklyn,047,BK73,North Side-South Side,2.89e+07,28149.97,"POLYGON ((-73.95814 40.72441, -73.95772 40.724...",BK73,North Side-South Side,7.0,3.57,26398.0,2.65,0,NaN
192,193,3,Brooklyn,047,BK90,East Williamsburg,3.91e+07,49246.97,"POLYGON ((-73.92406 40.71412, -73.92404 40.714...",BK90,East Williamsburg,4.0,3.00,10907.0,3.67,0,NaN
193,194,1,Manhattan,061,MN21,Gramercy,7.53e+06,12096.84,"POLYGON ((-73.97854 40.73690, -73.97858 40.736...",MN21,Gramercy,7.0,3.29,48246.0,1.45,0,NaN


In [165]:
cen_tract_nta_final_geo.to_file("cen_tract_nta_final_geo.json", driver="GeoJSON")